<a href="https://colab.research.google.com/github/paintedpotato/MI/blob/main/compressiveTs_daytrade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To use the CompressiveTransformer to classify day data for BTC/USD and execute trades based on the training set from 1 Jan 2022 to 1 Jan 2023, you will need to follow the steps outlined below:

Import the necessary libraries:

In [ ]:
pip install compressive_transformer_pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install quandl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

import quandl
import numpy as np
import torch
from compressive_transformer_pytorch import CompressiveTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier


In [ ]:
# activating the API key  
quandl.ApiConfig.api_key = "8daXuHDtVszfAfF_KanM"   

Use the Quandl API to fetch BTC/USD data for the desired time period:
Copy code


In [ ]:
# Get BTC/USD data from Quandl for the specified time period
data = quandl.get('BCHAIN/MKPRU', start_date="2022-01-01", end_date="2023-01-01")


In [ ]:
print(data)

               Value
Date                
2022-01-01  46249.56
2022-01-02  47763.33
2022-01-03  47327.87
2022-01-04  46441.99
2022-01-05  45862.98
...              ...
2022-12-28  16699.00
2022-12-29  16539.28
2022-12-30  16630.49
2022-12-31  16599.69
2023-01-01  16526.78

[366 rows x 1 columns]


Select the desired input features and create a label for whether the price increased or decreased:


In [ ]:
# Select the Close and Volume columns as input features
X = data[['Close', 'Volume']]

# Create a binary label for whether the price increased or decreased
y = np.where(data['Close'].shift(-1) > data['Close'], 1, 0)


KeyError: ignored

Create a transformer to apply the CompressiveTransformer to the input data:
Copy code


In [ ]:
# Create a transformer to apply the CompressiveTransformer to the input data
transformer = ColumnTransformer(
    [('compressive', CompressiveTransformer(), [0, 1])],
    remainder='passthrough'
)


Create a classifier and a pipeline:
Copy code


In [ ]:
# Create a classifier
classifier = RandomForestClassifier()

# Create a pipeline
pipeline = Pipeline([('transformer', transformer), ('classifier', classifier)])
Train the model on the training data:
Copy code
# Train the model
pipeline.fit(X, y)


Use the trained model to make predictions on new data and execute trades based on the predictions:
Copy code


In [ ]:
# Make a prediction on the last day of data
last_day = np.array([X.tail(1)]).reshape(1, -1)
prediction = pipeline.predict(last_day)[0]

# If the prediction is 1, buy BTC. If the prediction is 0, sell BTC.
if prediction == 1:
    print("Buy BTC")
else:
    print("Sell BTC")


This example uses the CompressiveTransformer to transform the 'Close' and 'Volume' columns of the BTC/USD data, and then trains a classifier (random forest) to predict whether the price will increase or decrease. Finally, it makes a prediction on the last day of data and prints whether to buy or sell BTC based on the prediction.